# Dekoratory

Ogólna idea - modyfikacja działania funkcji bez ingerowania w tę funkcję. Przykłady:
- dodanie mierzenia czasu
- dodanie logowania do pliku
- obsługa błędów

Często nie chcemy modyfikować kodu funkcji, chociażby dlatego, żeby nie zmniejszać czytelności. Albo jeśli pewne zmiany chcemy wprowadzić do wielu funkcji to zamiast poprawiać w każdej, lepiej napisać jedną modyfikację.

## Przykład 1 – dzielenie przez 0

In [1]:
def divide(a, b):
    return a / b

In [3]:
divide(6, 0)  # divide(6, 0) błąd

ZeroDivisionError: division by zero

Funkcja spełnia swoje zadanie, ale nie ma obsługi błędów. W takim razie modyfikujemy jej ciało

In [4]:
def divide(a, b):
    if b == 0:
        return None
    else:
        return a / b

In [6]:
divide(6, 0)

In [8]:
def modulo(a, b):
    return a % b

In [9]:
modulo(5, 2)

1

Modyfikacja obsługuje błąd, ale wymaga zmiany w funkcji. Zamiast tego, obsługę błędów można obsłużyć w innej funkcji

In [10]:
def divide(a, b):
    return a / b

Poniższe funkcja przyjmuje jako argument funkcję `divide` i zwraca jej zmodyfikowaną wersję.

In [14]:
def modify_division_function_to_handle_division_by_zero(divide_function): 
    def wrapper(a, b):  # wrapper to zmodyfikowana wersja funkcji
        if b == 0:
            return None
        else:
            return divide_function(a, b)  # nie duplikujemy kodu z funkcji dzielącej
    
    # Skąd w tym miejscu wiadomo czym jest a i b?
    # Nie wiadomo. po prostu zwracamy funkcję, która będzie miała takie parametry
    
    return wrapper  # zwraca ulepszoną funkcję

In [15]:
divide(6, 0)  # divide(6, 0) spowoduje błąd

ZeroDivisionError: division by zero

In [17]:
modify_division_function_to_handle_division_by_zero(divide)  # to zwraca wrapper

<function __main__.modify_division_function_to_handle_division_by_zero.<locals>.wrapper(a, b)>

In [18]:
modify_division_function_to_handle_division_by_zero(divide)(6, 3)

2.0

In [19]:
modify_division_function_to_handle_division_by_zero(divide)(6, 0)

Funkcji divide jako takiej nigdy nie chcemy wywoływać bo jest podatna na błąd

---

In [20]:
def handle_zero_division(divide_function):
    def wrapper(a, b):
        if b == 0:
            return None
        else:
            return divide_function(a, b)    

    return wrapper


@handle_zero_division
def divide(a, b):
    return a / b

In [21]:
divide

<function __main__.handle_zero_division.<locals>.wrapper(a, b)>

In [22]:
divide(6, 3)

2.0

In [23]:
divide(6, 0)

Powyższy zapis jest tak naprawdę wywołaniem funkcji `handle_zero_division` do której przekazujemy `divide`. To zwraca wrapper, do którego następnie przekazujemy argumenty. To co zwróci wrapper jest finalnym wynikiem powyższej linii.

Wydaje się że jest to strzelanie z armaty do wróbla, ale teraz tego walidatora możemy użyć również do innych funkcji, a nie tylko do tej jednej.

In [24]:
def handle_zero_division(func):    # func - zwykle tak piszemy
    def wrapper(*args, **kwargs):  # args, kwargs - jesteśmy w stanie obsłużyć sytuacje gdzie są inne argumenty
        if args[1] == 0:
            return None
        else:
            return func(*args, **kwargs)   # divide_function(a, b) 
    
    return wrapper


@handle_zero_division
def divide(a, b):
    return a / b


@handle_zero_division
def modulo(a, b):
    return a % b

In [26]:
divide(6, 0)

In [27]:
modulo(6, 0)

## Przykład 2 – transformacja wartości zwracanej przez funkcję

In [28]:
def return_argument(x):
    return x

In [29]:
return_argument(10)

10

In [33]:
def square(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** 2

    return wrapper

In [34]:
@square
def return_argument(x):
    return x

In [36]:
return_argument(10)

100

## Przykład 3 – bardziej abstrakcyjny, obrazuje ideę dekoratora

In [37]:
"hello".upper()

'HELLO'

In [39]:
str.upper  #("hello")

<method 'upper' of 'str' objects>

In [44]:
def my_decorator(func):
    return str.upper



@my_decorator
def my_function(x):
    return x+1+2*4

Pytanie: co zwroci wywołanie funkcji?

In [45]:
my_function

<method 'upper' of 'str' objects>

In [43]:
my_function("hello")

'HELLO'

> **ZADANIA**

## Wiele dekoratorów – przykład 1

In [56]:
import time


def validate_division(func):
    def wrapper(*args, **kwargs):
        if args[1] == 0:
            return None
        else:
            return func(*args, **kwargs)
    
    return wrapper


def measure_time(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        stop = time.perf_counter()
        print("Czas: ", stop - start)

        return result

    return wrapper


def pure_divide(a, b):
    return a / b


@measure_time
@validate_division
def divide(a, b):
    return a / b

In [57]:
divide(10, 2) # is None

Czas:  4.1679886635392904e-06


5.0

In [59]:
measure_time(validate_division(pure_divide))(10, 20)

Czas:  2.7269998099654913e-06


0.5

---

In [60]:
@validate_division
@measure_time
def divide(a, b):
    return a / b

In [61]:
divide(10, 2)

Czas:  3.156979801133275e-06


5.0

In [62]:
validate_division(measure_time(pure_divide))(10, 20)

Czas:  2.3139873519539833e-06


0.5

Nie ma printowania czasu, ponieważ tutaj najpierw wywołujemy funkcję walidującą, a ona po sprawdzeniu czy b==0 od razu zwraca None zamiast wywoływać kolejne funkcje.

Kiedy b!=0, czas będzie printowany.

In [ ]:
divide(10, 1)

## Wiele dekoratorów – przykład 2

In [63]:
def add_exclamation(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result += "!"
        return result
    return wrapper


def add_quotation(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result += "?"
        return result
    return wrapper


@add_exclamation
@add_quotation
def make_upper(text):
    return text.upper()

In [64]:
make_upper("hello")

'HELLO?!'

Wołamy add_exclamation, które woła add_quotation, które woła make_upper

> **ZADANIA**

## Parametryzacja dekoratora

In [66]:
round(1.4232, 2)

1.42

In [69]:
def round_result(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        rounded_result = round(result)
        return rounded_result

    return wrapper



@round_result
def calculate_average(a, b):
    return (a + b) / 2


calculate_average(1.242342, 2.23)

2

---

In [71]:
def actual_rounding_decorator(param):
    def round_result(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            rounded_result = round(result, param)
            return rounded_result

        return wrapper
    return round_result


@actual_rounding_decorator(2)
def calculate_average(a, b):
    return (a + b) / 2


calculate_average(1.242342, 2.23)

1.74

In [ ]:
actual_rounding_decorator(round_result(calculate_average))